# Домашнее задание для Junior DS по разработке PD модели

<b>Задание:</b> Необходимо разработать модель предсказания дефолта. Даны финансовые данные 32 395 компаний (выручка, активы, пассивы и т.д). Результат будет оцениваться на скрытой выборке из 200 компаний по метрике качества accuracy_score

<b>Цель задания</b> научиться / показать знания в разработке моделей оценки риска - probability of default. Задание приближено к реальному, данные синтетические. Лидерборд для оценки собственного решения и геймификации процесса обучения. Первое место в лидерборде не является целью. Цель - правильно разработанная модель, удовлетворяющая требованиям банковской системы. В случае возникновения вопросов, сотрудники блока Риски готовы проконсультировать и дать обратную связь на модель (телеграм чат).

<b>Метрика качества</b> accuracy_score вычисляется, как ( True positive + True negative ) / Total. Если сабмит-файл, неправильного формата или неверное количество строк, метрика будет = 0.

### Импортируем библиотеки

In [46]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

## Обработка данных


### Загрузка данных

In [2]:
train_df = pd.read_csv('https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-train.csv', sep=';')
test_df = pd.read_csv('https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-test.csv', sep=';')
desc_df = pd.read_csv('https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-desc.csv', sep=';')

### Первычный анализ

In [3]:
train_df.head(10)

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,ar_other_profit_and_losses,ar_net_profit,ab_immobilized_assets,ab_mobile_current_assets,ab_inventory,ab_accounts_receivable,ab_other_current_assets,ab_cash_and_securities,ab_losses,ab_own_capital,ab_borrowed_capital,ab_long_term_liabilities,ab_short_term_borrowing,ab_accounts_payable,ab_other_borrowings,bus_age,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg,default_12m
0,7,52996000.0,48344000.0,48344000.0,0.0,0.0,4652000.0,-2608000.0,2044000.0,846000.0,0.0,1198000.0,0.0,7240000.0,4988000.0,2252000.0,0.0,255000.0,1198000.0,663000.0,15872000.0,0.0,11349000.0,4523000.0,0.0,198,136,3,3,3,[1-100],50000.0,2,0,0,0,0
1,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256,135,12,12,12,[1-100],10000.0,5,0,0,0,0
2,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33,33,28,28,28,[1-100],10000.0,2,0,0,0,0
3,22,32049000.0,29568000.0,29568000.0,0.0,0.0,2481000.0,-260000.0,2221000.0,302000.0,0.0,1919000.0,0.0,4245000.0,4082000.0,163000.0,0.0,25000.0,1919000.0,2219000.0,3652000.0,0.0,3652000.0,0.0,0.0,185,139,8,8,8,[1-100],300000.0,1,0,0,0,0
4,23,16458000.0,14935000.0,14935000.0,0.0,0.0,1523000.0,-274000.0,1249000.0,538000.0,0.0,711000.0,0.0,8950000.0,8606000.0,344000.0,0.0,492000.0,711000.0,12439000.0,1279000.0,0.0,0.0,1279000.0,0.0,220,138,27,27,27,[1-100],10000.0,2,0,0,0,0
5,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,100,5,5,5,[1-100],350000.0,1,0,0,0,0
6,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,266,27,7,7,7,[1-100],35000.0,3,0,0,0,0
7,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31,31,30,30,30,[1-100],10000.0,2,0,0,0,0
8,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27,27,26,26,26,[1-100],10000.0,1,0,0,0,0
9,40,125432000.0,122655000.0,122655000.0,0.0,0.0,2777000.0,-63000.0,2714000.0,579000.0,0.0,2135000.0,0.0,24680000.0,8263000.0,16417000.0,0.0,106000.0,2135000.0,11267000.0,11912000.0,0.0,13000.0,11899000.0,0.0,170,132,15,15,15,(100-500],10000.0,2,0,0,0,0


Рассмотрим описание колонок во второй таблице:

In [4]:
desc_df.head(50)

,field,desc_eng,desc_rus
0,ar_revenue,Sales revenue (at the end of the last year),Выручка от реализации (на конец последнего года)
1,ar_total_expenses,Total expenses for the last year,Общие расходы за последний год
2,ar_sale_cost,Cost of sales (at the end of the last year),Себестоимость продаж (на конец последнего года)
3,ar_selling_expenses,Selling expenses (at the end of the last year),Коммерческие расходы (на конец последнего года)
4,ar_management_expenses,Administrative expenses (at the end of the las...,Управленческие расходы (на конец последнего года)
5,ar_sale_profit,Profit from sales (at the end of the last year),Прибыль от реализации (на конец последнего года)
6,ar_balance_of_rvns_and_expns,Balance of income and expenses (at the end of ...,Баланс доходов и расходов (на конец последнего...
7,ar_profit_before_tax,Gross Profit (end of last year),Валовая прибыль (на конец последнего года)
8,ar_taxes,Current income tax (at the end of the last year),Текущий на налог на прибыль (на конец последне...
9,ar_other_profit_and_losses,Other income and expenses (at the end of the l...,Прочие доходы и расходы (на конец последнего г...


Признаки отражают финансовую отчетность компании. На основе этих признаков мы сможем расчитать финансовые мультипликаторы, которые будут яснее отображать финансовую устойчивость компании, а, соотсветственно, и лучше демонстрировать вероятность дефолта. 

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32395 entries, 0 to 32394
Data columns (total 37 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   record_id                     32395 non-null  int64  
 1   ar_revenue                    17250 non-null  float64
 2   ar_total_expenses             17250 non-null  float64
 3   ar_sale_cost                  17250 non-null  float64
 4   ar_selling_expenses           17250 non-null  float64
 5   ar_management_expenses        17250 non-null  float64
 6   ar_sale_profit                17250 non-null  float64
 7   ar_balance_of_rvns_and_expns  17250 non-null  float64
 8   ar_profit_before_tax          17250 non-null  float64
 9   ar_taxes                      17250 non-null  float64
 10  ar_other_profit_and_losses    17250 non-null  float64
 11  ar_net_profit                 17250 non-null  float64
 12  ab_immobilized_assets         17250 non-null  float64
 13  a

У нас есть 1 категориальный признак. Это количество сотрудников.
Можем закодировать через LabelEncoding, тк видна четкая иерархия. 

In [6]:
train_df['ul_staff_range'].value_counts()

[1-100]      29281
(100-500]     2757
> 500          357
Name: ul_staff_range, dtype: int64

In [7]:
le = preprocessing.LabelEncoder()
le.fit(train_df['ul_staff_range'])
train_df['ul_staff_range'] = le.transform(train_df['ul_staff_range'])
test_df['ul_staff_range'] = le.transform(test_df['ul_staff_range'])

Рассмотрим уникальные значения у каждого признака.

In [8]:
train_df.nunique()

record_id                       32395
ar_revenue                      12893
ar_total_expenses               12779
ar_sale_cost                    12664
ar_selling_expenses              1994
ar_management_expenses           1093
ar_sale_profit                   7163
ar_balance_of_rvns_and_expns     5063
ar_profit_before_tax             6526
ar_taxes                         2247
ar_other_profit_and_losses        595
ar_net_profit                    6231
ab_immobilized_assets            3498
ab_mobile_current_assets        11222
ab_inventory                     9143
ab_accounts_receivable           8283
ab_other_current_assets          1210
ab_cash_and_securities           3677
ab_losses                        6175
ab_own_capital                   9090
ab_borrowed_capital             10261
ab_long_term_liabilities         2294
ab_short_term_borrowing          4896
ab_accounts_payable              8797
ab_other_borrowings              1437
bus_age                           315
ogrn_age    

Видим, что признак ul_systematizing_flg имеет 1 уникальное значение. Удалим его. Также удалим record_id так как он не несет информативности для нашей задачи.

In [9]:
train_df = train_df.drop(['record_id','ul_systematizing_flg'],axis = 1)

### Обработка пропусков, выбросов и дубликатов

Мы предполагаем что в train_df пропуски являются частью *предметной области*. Компании с пропусками просто не имеют/не предоставили своих бухгалтерских отчетов.

Пропуски в train_df составляют 46% выборки, а в test_df 59% выборки. Более того, в train_df 2% дефолтов произошло с компаниями без пропусков, а 4% дефолтов с компаниями с пропусками. Это слишком большая потеря данных.

Также заметим, что можно выделить две примерно одинаковые подвыборки: с финансовой отчетностью и без. Так как эти подвыборки очень различаются по своим вводным, будем рассматривать их раздельно.


In [10]:
target = train_df['default_12m']

print(f'{np.round((1-len(train_df.dropna()) / len(train_df)) * 100, 3)}% строк в train_df имеют пропуски')
print(f'{np.round((1-len(test_df.dropna()) / len(test_df))* 100, 3)}% строк в test_df имеют пропуски')

print()

print(f'{np.round((train_df.dropna()["default_12m"].sum() / train_df["default_12m"].count()) * 100, 3)}\
% строк c пропусками из train_df объявили дефолт')

a = pd.concat([train_df, train_df.dropna()]).drop_duplicates(keep=False)['default_12m']
print(f'{np.round((a.sum()/train_df["default_12m"].count())*100, 3)}% строк без пропусков из train_df объявили дефолт')
print(f'{np.round((target.sum()/train_df["default_12m"].count())*100, 3)}% всего строк из train_df объявили дефолт')

46.751% строк в train_df имеют пропуски
59.0% строк в test_df имеют пропуски

2.457% строк c пропусками из train_df объявили дефолт
2.806% строк без пропусков из train_df объявили дефолт
6.455% всего строк из train_df объявили дефолт


Подвыборки в трейне:

In [11]:
pd.DataFrame(
    train_df
    .isnull()
    .T
    .sum(), columns = ["Кол-во пропусков"]
    )\
    .reset_index()\
    .groupby("Кол-во пропусков")\
    .count()

,index
Кол-во пропусков,
0,17250
24,15145


In [12]:
sum_null_in_object = pd.DataFrame(train_df.isnull().T.sum(),columns = ['count_nans'])
ids_with_24_nans = sum_null_in_object.query('count_nans == 24').index
train_df.loc[ids_with_24_nans,:][:3]

,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,ar_other_profit_and_losses,ar_net_profit,ab_immobilized_assets,ab_mobile_current_assets,ab_inventory,ab_accounts_receivable,ab_other_current_assets,ab_cash_and_securities,ab_losses,ab_own_capital,ab_borrowed_capital,ab_long_term_liabilities,ab_short_term_borrowing,ab_accounts_payable,ab_other_borrowings,bus_age,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,default_12m
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256,135,12,12,12,2,10000.0,5,0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33,33,28,28,28,2,10000.0,2,0,0,0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,100,5,5,5,2,350000.0,1,0,0,0


Подвыборки в тесте также присутствуют:

In [13]:
pd.DataFrame(
    test_df
    .isnull()
    .T
    .sum(), columns = ["Кол-во пропусков"]
    )\
    .reset_index()\
    .groupby("Кол-во пропусков")\
    .count()

,index
Кол-во пропусков,
0,82
24,118


Существует немало компаний с одинаковыми значениями в статьях баланса. Более того на частных примерах мы увидили, что компании с одинаковыми значениями в статьях баланса по остальным признакам имеют минимальные различия.

Будем считать эти строки дубликатами и удалим их из выборки.

In [44]:
print(f'{np.round(train_df.duplicated().sum()/len(train_df), 3)}% выборки train_df полные дубликаты')

a = pd.concat([train_df, train_df.dropna()]).drop_duplicates(keep=False)

print(f'{np.round(100 * a.iloc[:,:24].duplicated().sum()/len(train_df), 3)}\
% выборки train_df дубликаты по статьям расхода')

print(f'{np.round(100 * train_df.iloc[:,24:].duplicated().sum()/len(train_df), 3)}\
% выборки train_df дубликаты по НЕ статьям расхода')

train_df = train_df.drop_duplicates() 

0.0% выборки train_df полные дубликаты
38.413% выборки train_df дубликаты по статьям расхода
15.878% выборки train_df дубликаты по НЕ статьям расхода


Мы заметили, что в некторых статьях есть отрицательные значения. По МСФО таких статей не должно существовать. Более того, таких компаний немного и они не объявляли дефолт. Однако если выбросить все отрицательные наблюдения, то мы теряем 87% выборки.

In [16]:
def remove_all_negs(df = train_df):
    k = df.min()
    return remove_cols_negs(df = train_df, cols=k[k<0].keys())

def remove_cols_negs(df = train_df, cols=[1*'ar_total_expenses',
                                              1*'ar_sale_cost',
                                              1*'ar_selling_expenses',
                                              1*'ar_management_expenses',
                                              0*'ar_sale_profit',
                                              0*'ar_balance_of_rvns_and_expns',
                                              0*'ar_profit_before_tax',
                                              1*'ar_taxes',
                                              0*'ar_other_profit_and_losses',
                                              0*'ar_net_profit',
                                              1*'ab_accounts_receivable',
                                              1*'ab_other_current_assets',
                                              1*'ab_losses',
                                              1*'ab_own_capital',
                                              1*'ab_long_term_liabilities',
                                              1*'ab_short_term_borrowing',
                                              1*'ab_other_borrowings']):
    for column_with_negs in cols:
        if column_with_negs == '':
            continue
        df = df[(train_df[column_with_negs]>=0) | (train_df[column_with_negs].isna())]
    return df

In [47]:
print(f'Если убрать ar_total_expenses, теряется {np.round(100-len(remove_cols_negs())/len(train_df)*100, 3)}% выборки')
print(f'Если убрать все отрицательные значения, теряется {np.round(100-len(remove_all_negs())/len(train_df)*100, 3)}% выборки')

Если убрать ar_total_expenses, теряется 7.71% выборки
Если убрать все отрицательные значения, теряется 47.352% выборки


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/li

Разделим вышезамеченные выборки с отчетностью и без. Мы видим, что разбиение по Nan значениям бинарное. Проверим также на всякий случай точно ли эти пропуски делятся по отсутствию отчетности.

In [18]:
train_df.isna().sum()

ar_revenue                      10215
ar_total_expenses               10215
ar_sale_cost                    10215
ar_selling_expenses             10215
ar_management_expenses          10215
ar_sale_profit                  10215
ar_balance_of_rvns_and_expns    10215
ar_profit_before_tax            10215
ar_taxes                        10215
ar_other_profit_and_losses      10215
ar_net_profit                   10215
ab_immobilized_assets           10215
ab_mobile_current_assets        10215
ab_inventory                    10215
ab_accounts_receivable          10215
ab_other_current_assets         10215
ab_cash_and_securities          10215
ab_losses                       10215
ab_own_capital                  10215
ab_borrowed_capital             10215
ab_long_term_liabilities        10215
ab_short_term_borrowing         10215
ab_accounts_payable             10215
ab_other_borrowings             10215
bus_age                             0
ogrn_age                            0
adr_actual_a

То же число! Невероятно!

In [19]:
train_df['ar_revenue'].isnull().sum()

10215

Делим:

In [20]:
def sep_df_byfin(df):
    df_nofin = get_df_nofin(df)
    df_fin   = get_df_fin(df)
    return df_fin, df_nofin
    
def get_df_fin(df):
    return df.dropna()

def get_df_nofin(df):
    return pd.concat([df, df.dropna()]).drop_duplicates(keep=False)

df_fin_train, df_no_fin_train = sep_df_byfin(df = train_df)
df_fin_test, df_no_fin_test = sep_df_byfin(df = test_df)

print(f'{np.round(100*len(df_fin_train)/len(train_df), 3)}% компаний имеют фин. отчетность в трейне')
print(f'{np.round(100*len(df_no_fin_train)/len(train_df), 3)}% компаний НЕ имеют фин. отчетность в трейне')
print()
print(f'{np.round(100*len(df_fin_test)/len(test_df), 3)}% компаний имеют фин. отчетность в тесте')
print(f'{np.round(100*len(df_no_fin_test)/len(test_df), 3)}% компаний НЕ имеют фин. отчетность в тесте')

61.583% компаний имеют фин. отчетность в трейне
38.417% компаний НЕ имеют фин. отчетность в трейне

41.0% компаний имеют фин. отчетность в тесте
59.0% компаний НЕ имеют фин. отчетность в тесте


В тесте есть дубликаты:

In [21]:
len(df_no_fin_test) + len(df_fin_test)

200

In [22]:
len(test_df)

200

In [23]:
pd.concat([test_df, df_no_fin_test.append(df_fin_test)]).drop_duplicates(keep=False)

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,ar_other_profit_and_losses,ar_net_profit,ab_immobilized_assets,ab_mobile_current_assets,ab_inventory,ab_accounts_receivable,ab_other_current_assets,ab_cash_and_securities,ab_losses,ab_own_capital,ab_borrowed_capital,ab_long_term_liabilities,ab_short_term_borrowing,ab_accounts_payable,ab_other_borrowings,bus_age,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg


### Баланс *классов*

Заметим, что у классов в обоих выборках наблюдается достаточно серьезный дисбаланс. Можем обработать несколькими способами: 

*   Oversampling
*   Undersampling


In [28]:
print(f'{np.round(df_fin_train["default_12m"].values.mean() * 100, 3)}% дефолт у компаний с отчетностью')
print(f'{np.round(df_no_fin_train["default_12m"].values.mean() * 100, 3)}% дефолт у компаний без отчетности')

4.739% дефолт у компаний с отчетностью
10.23% дефолт у компаний без отчетности


In [30]:
df_no_fin_train[df_no_fin_train['default_12m'] == 1]['default_12m'].sum()

1045

In [32]:
def undersampling(df):

  one_class = df[df['default_12m'] == 1]['default_12m'].sum()
  df_class_0 = df[
                 df['default_12m'] == 0
                 ]\
                 .sample(one_class, random_state=42)

  df_undersampling = df_class_0.append(
      df[
         df['default_12m'] == 1
         ]).sample(frac=1)

  return df_undersampling


def oversampling(x_train, y_train):
  x_train_sm, y_train_sm = SMOTE(random_state=12).fit_resample(x_train, y_train)
  return x_train_sm, y_train_sm 


## Feature Engineering

[ТУТ ВСТАВИТЬ ОПИСАНИЕ ВСЕХ ПРИЗНАКОВ]


In [33]:
def make_features(df):
    frame = df.copy()
    # Доля дохода, которая учитывает пост-оплату
    frame['frac_post_pay'] = frame['ab_accounts_receivable']/frame['ar_revenue']
    frame['OPEX'] = frame['ar_total_expenses'] - frame['ar_selling_expenses'] - frame["ar_management_expenses"]
    # ДОля операционных расходов
    frame['OER'] = frame['OPEX']/frame['ar_revenue']
    frame['frac_comer_exp'] = frame['ar_selling_expenses' ]/ frame['ar_total_expenses']
    frame['Net_margin'] = frame['ar_net_profit']/frame['ar_revenue']
    frame['gross_profit_margin'] = (frame['ar_revenue'] - frame['ar_sale_cost'])/frame['ar_revenue']
    frame['OP_Margin'] = frame['OPEX']/frame['ar_revenue']
    frame['ROE'] = frame['ar_net_profit']/frame['ab_own_capital']
    frame['ROA'] = frame['ar_net_profit']/(frame['ab_own_capital'] + frame['ab_borrowed_capital'])
    frame['Debt/EBIT'] = (frame['ab_long_term_liabilities'] + frame['ab_other_borrowings'] + frame['ab_short_term_borrowing'])/frame['ar_profit_before_tax']
    frame['Debt_ratio'] = (frame['ab_long_term_liabilities'] + frame['ab_other_borrowings'] + frame['ab_short_term_borrowing'])/(frame['ab_own_capital'] + frame['ab_borrowed_capital'])
    frame['tax_ratio'] = frame['ar_taxes']/frame['ar_profit_before_tax']
    frame['time_gap'] = frame['ab_accounts_payable']/(frame['ab_cash_and_securities']+frame['ab_accounts_receivable'])
    frame['borrowing_balance'] = frame['ab_accounts_receivable']/frame['ab_accounts_payable']
    frame['Debt/Equity'] = (frame['ab_long_term_liabilities'] + frame['ab_other_borrowings'] + frame['ab_short_term_borrowing'])/frame['ab_own_capital']
    frame['current_ratio'] = (frame['ab_mobile_current_assets']+frame['ab_other_current_assets'])/(frame['ab_short_term_borrowing']+frame['ab_other_borrowings'])
    frame['cash_ratio'] = frame['ab_cash_and_securities']/frame['ab_short_term_borrowing']
    frame['fast_pay'] = (frame['ab_cash_and_securities']+frame['ab_accounts_receivable'])/(frame['ab_own_capital'] + frame['ab_borrowed_capital'])
    frame['tumover_ratio'] = frame['ar_revenue']/(frame['ab_own_capital'] + frame['ab_borrowed_capital'])
    return frame

Создаем признаки только для выборок с финансовой отчетностью:

In [34]:
df_fin_train_feature = make_features(df_fin_train)
df_fin_test_feature = make_features(df_fin_test)

Теперь же очистим наши выборки от ненужных столбцов. Для _part_2 это все абсолютные значения финансовых показателей. А для _part_1 это все переменные отвечающие за финансовую отчетность + ul_strategic_flg

Уберем старые признаки:


*   Для выборок с фин. отчетностью - это абсолютные значения финансовых показателей
*   Для выборок без фин. отчетность - это все столбцы с NaN значениями


In [35]:
fin_abs_features = ['ar_revenue',
                   'ar_other_profit_and_losses',
                   'ar_profit_before_tax', 
                   'ar_taxes', 
                   'ar_total_expenses', 
                   'ar_sale_cost', 
                   'ar_selling_expenses', 
                   'ar_management_expenses',
                   'ar_sale_profit', 
                   'ar_balance_of_rvns_and_expns']

In [36]:
# Выборки с фин отчетностью
df_fin_train_feature_complex = df_fin_train_feature.drop(fin_abs_features, axis = 1)
df_fin_test_feature_complex = df_fin_test_feature.drop(fin_abs_features, axis = 1)

# Выборки без фин отчетности
df_no_fin_train_dropna = df_no_fin_train.dropna(axis=1,how='all')
df_no_fin_test_dropna = df_no_fin_test.dropna(axis=1,how='all')

Посмотрим на новые признаки у выборок с финансовой отчетностью:

In [37]:
df_fin_train_feature_complex.head(10)

,ar_net_profit,ab_immobilized_assets,ab_mobile_current_assets,ab_inventory,ab_accounts_receivable,ab_other_current_assets,ab_cash_and_securities,ab_losses,ab_own_capital,ab_borrowed_capital,ab_long_term_liabilities,ab_short_term_borrowing,ab_accounts_payable,ab_other_borrowings,bus_age,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,default_12m,frac_post_pay,OPEX,OER,frac_comer_exp,Net_margin,gross_profit_margin,OP_Margin,ROE,ROA,Debt/EBIT,Debt_ratio,tax_ratio,time_gap,borrowing_balance,Debt/Equity,current_ratio,cash_ratio,fast_pay,tumover_ratio
0,1198000.0,0.0,7240000.0,4988000.0,2252000.0,0.0,255000.0,1198000.0,663000.0,15872000.0,0.0,11349000.0,4523000.0,0.0,198,136,3,3,3,2,50000.0,2,0,0,0,0.042494,48344000.0,0.912220,0.000000,0.022605,0.087780,0.912220,1.806938,0.072452,5.552348,0.686362,0.413894,1.804148,0.497900,17.117647,0.637942,0.022469,0.151618,3.205080
3,1919000.0,0.0,4245000.0,4082000.0,163000.0,0.0,25000.0,1919000.0,2219000.0,3652000.0,0.0,3652000.0,0.0,0.0,185,139,8,8,8,2,300000.0,1,0,0,0,0.005086,29568000.0,0.922587,0.000000,0.059877,0.077413,0.922587,0.864804,0.326861,1.644304,0.622041,0.135975,0.000000,inf,1.645786,1.162377,0.006846,0.032022,5.458866
4,711000.0,0.0,8950000.0,8606000.0,344000.0,0.0,492000.0,711000.0,12439000.0,1279000.0,0.0,0.0,1279000.0,0.0,220,138,27,27,27,2,10000.0,2,0,0,0,0.020902,14935000.0,0.907461,0.000000,0.043201,0.092539,0.907461,0.057159,0.051830,0.000000,0.000000,0.430745,1.529904,0.268960,0.000000,inf,inf,0.060942,1.199738
9,2135000.0,0.0,24680000.0,8263000.0,16417000.0,0.0,106000.0,2135000.0,11267000.0,11912000.0,0.0,13000.0,11899000.0,0.0,170,132,15,15,15,0,10000.0,2,0,0,0,0.130884,122655000.0,0.977861,0.000000,0.017021,0.022139,0.977861,0.189491,0.092109,0.004790,0.000561,0.213338,0.720148,1.379696,0.001154,1898.461538,8.153846,0.712844,5.411450
10,104000.0,0.0,38004000.0,23189000.0,14815000.0,0.0,449000.0,104000.0,10000.0,38443000.0,0.0,4986000.0,33457000.0,0.0,29,29,4,4,4,2,10000.0,2,0,0,0,0.248699,59331000.0,0.995988,0.000000,0.001746,0.004012,0.995988,10.400000,0.002705,35.112676,0.129665,0.267606,2.191889,0.442807,498.600000,7.622142,0.090052,0.396952,1.549164
11,21608000.0,2094000.0,106180000.0,1180000.0,105000000.0,0.0,557000.0,21608000.0,90313000.0,18518000.0,0.0,7500000.0,11018000.0,0.0,102,102,4,4,4,1,300000.0,2,0,0,0,0.582929,142075000.0,0.788758,0.103724,0.119961,0.211242,0.788758,0.239257,0.198546,0.347094,0.068914,0.000000,0.104380,9.529860,0.083045,14.157333,0.074267,0.969917,1.655089
12,71000.0,1269000.0,35349000.0,28034000.0,7237000.0,78000.0,2008000.0,71000.0,94000.0,38532000.0,2866000.0,0.0,35666000.0,0.0,24,24,7,7,7,2,21000.0,1,0,0,0,0.204580,4710000.0,0.133145,0.830635,0.002007,0.866855,0.133145,0.755319,0.001838,32.202247,0.074199,0.202247,3.857869,0.202910,30.489362,inf,inf,0.239347,0.915834
14,855000.0,0.0,11564000.0,8770000.0,2794000.0,0.0,606000.0,855000.0,865000.0,0.0,0.0,0.0,0.0,0.0,20,20,16,16,16,2,10000.0,1,0,0,0,0.120733,22064000.0,0.953418,0.000000,0.036946,0.046582,0.953418,0.988439,0.988439,0.000000,0.000000,0.200187,0.000000,inf,0.000000,inf,inf,3.930636,26.753757
15,157000.0,0.0,3879000.0,1643000.0,2236000.0,0.0,213000.0,157000.0,219000.0,3873000.0,0.0,2387000.0,1486000.0,0.0,48,48,16,16,16,2,10000.0,1,0,0,0,0.138469,15893000.0,0.984209,0.000000,0.009723,0.015791,0.984209,0.716895,0.038368,12.116751,0.583333,0.203046,0.606778,1.504711,10.899543,1.625052,0.089233,0.598485,3.946237
16,0.0,10000.0,0.0,0.0,0.0,0.0,150000.0,0.0,10000.0,150000.0,0.0,150000.0,0.0,0.0,20,20,4,4,4,2,10000.0,1,0,0,0,0.000000,146000.0,0.169767,0.182482,0.000000,0.830233,0.169767,0.000000,0.000000,inf,0.937500,NaN,0.000000,NaN,15.000000,0.000000,1.000000,0.937500,5.375000


Заметим, что в некоторых новых признаках присутствуют значения inf и nan (из-за деления на 0). Заменим их на 0: **(точно?)**

In [38]:
df_fin_train_feature_complex_fillna = df_fin_train_feature_complex.fillna(0)
df_fin_test_feature_complex_fillna = df_fin_test_feature_complex.fillna(0)

df_fin_train_feature_complex_fillna = df_fin_train_feature_complex_fillna.replace(np.inf,0)
df_fin_test_feature_complex_fillna = df_fin_test_feature_complex_fillna.replace(np.inf,0)
df_fin_train_feature_complex_fillna = df_fin_train_feature_complex_fillna.replace(-np.inf,0)
df_fin_test_feature_complex_fillna = df_fin_test_feature_complex_fillna.replace(-np.inf,0)

[НУЖНЫ ОБЪЯСНЕНИЯ]

In [39]:
def filter_function(X_train, y_train, column_target):
    """
    Функция вычисления t статистики
    """
    grouped = X_train.merge(
        y_train,left_index=True,right_index=True
        ).groupby(column_target)
    
    mean_neg = grouped.mean().iloc[0]
    mean_pos = grouped.mean().iloc[1]

    std_neg = grouped.std().iloc[0]
    std_pos = grouped.std().iloc[1]

    count_neg = grouped.count().iloc[0]
    count_pos = grouped.count().iloc[1]
    
    t = np.abs(mean_pos - mean_neg)/np.sqrt((count_pos*std_pos**2 + count_neg*std_neg**2)/(count_pos+count_neg))
    return t

In [40]:
X_train_fin = df_fin_train_feature_complex_fillna.drop('default_12m', axis = 1) 
y_train_fin = df_fin_train_feature_complex_fillna['default_12m']

X_train_no_fin = df_no_fin_train_dropna.drop('default_12m',axis = 1) 
y_train_no_fin = df_no_fin_train_dropna['default_12m']

In [41]:
top_features_fin = pd.DataFrame(data = filter_function(X_train_fin, 
                                                          y_train_fin,
                                                          'default_12m')
                                                          .to_dict()
                                                          .items())\
                                                          .sort_values(by = 1, ascending=False)\
                                                          .reset_index(drop = True)[:25][0]\
                                                          .tolist()

top_features_no_fin = pd.DataFrame(data = filter_function(X_train_no_fin, 
                                                          y_train_no_fin,
                                                          'default_12m')
                                                          .to_dict()
                                                          .items())\
                                                          .sort_values(by = 1, ascending=False)\
                                                          .reset_index(drop = True)[:7][0]\
                                                          .tolist()

## Отбор признаков

### WOE

## Модель

[ДОБАВИТЬ БУКОВ]

In [ ]:
steps = [
         ('smote',SMOTE(random_state=42)),
         ('stand',StandardScaler()), 
         ('model', LogisticRegression(max_iter=1000))
         ]

LGREG = Pipeline(steps=steps)

# Зададим диапазон перебора параметра
parametrs_logreg = {'model__C': np.logspace(0, -5, 20),
                    'smote__k_neighbors': range(2, 12)}

# parametrs_Logreg = {'model__C':[0.0006951927961775605],'smote__k_neighbors':[9]}


# Инициализируем кросс-валидационный перебор параметра
grid_logreg_no_fin = GridSearchCV(LGREG,
                                  parametrs_logreg,
                                  scoring='accuracy'
                                  )

# Обучим модель
grid_logreg_no_fin.fit(X_train_no_fin.loc[:,top_features_no_fin],
                       y_train_no_fin)

In [ ]:
grid_logreg_no_fin.best_score_

In [ ]:
grid_logreg_no_fin.best_params_

In [ ]:
y_train_no_fin.value_counts(normalize=True)

In [ ]:
steps = [
         ('smote', SMOTE(random_state=42)),
         ('stand', StandardScaler()), 
         ('model', LogisticRegression(max_iter=1000))
         ]

LGREG = Pipeline(steps=steps)

# Зададим диапазон перебора параметра
parametrs_logreg = {'model__C':np.logspace(0,-5,20),
                    'smote__k_neighbors':range(2,12)}

# Инициализируем кросс-валидационный перебор параметра
grid_logreg_fin= GridSearchCV(LGREG, 
                              parametrs_logreg, 
                              scoring='accuracy')

# Обучим модель
grid_logreg_part_1.fit(X_train_fin.loc[:, top_features_fin],
                       y_train_fin)

In [ ]:
grid_logreg_part_1.best_score_

In [ ]:
grid_logreg_part_1.best_params_

### Выгрузка результата


In [ ]:
preds_no_fin = grid_logreg_no_fin.predict_proba(df_no_fin_train_dropna.loc[:,top_features_no_fin])

preds_fin = grid_Logreg_fin.predict_proba(df_fin_train_feature_complex_fillna.loc[:,top_features_fin])

In [ ]:
# ДОДЕЛАТЬ (просто вытянуть по индексу id)

# answ_fin = pd.DataFrame()
# answ_fin['id'] = test_records_fin
# answ_fin'predict'] = preds_fin

# answ_no_fin = pd.DataFrame()
# answ_no_fin['id'] = test_records_no_fin
# answ_no_fin['predict'] = preds_no_fin

# answ_df = pd.concat([answ_no_fin,answ_fin],sort=False)

In [ ]:
answ_df = answ_df.reset_index(drop = True)

In [ ]:
answ_df.to_csv('PD-submit_features_low_thr.csv',index=False, sep=';')

## Рекомендации